In [ ]:
"""
Created on Wed Oct 11 12:29 2023

Look at relationship between input T and predicted melt

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import multimelt.melt_functions as meltf
import basal_melt_neural_networks.data_formatting as dfmt
from scipy.stats import kde

In [ ]:
%matplotlib qt5

In [ ]:
nemo_run = 'bf663'

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
inputpath_profiles='/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_'+nemo_run+'/'
inputpath_plumes='/bettik/burgardc/DATA/NN_PARAM/interim/PLUMES/SMITH_'+nemo_run+'/'
inputpath_boxes='/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'


outputpath_melt = '/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
outputpath_nn = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/SMITH_'+nemo_run+'_EXTRAPDRAFT_CHUNKS/'
inputpath_tides = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/TIDES/'

outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'/'


In [ ]:
mod_size = 'small' #'mini', 'small', 'medium', 'large', 'extra_large'
TS_opt = 'extrap' # extrap, whole, thermocline
norm_method =  'std' # std, interquart, minmax
exp_name = 'newbasic2' #'allbutconstants' #'onlyTSdraftandslope' #'TSdraftbotandiceddandwcd' #'onlyTSisfdraft' #'TSdraftbotandiceddandwcdreldGL' #TSdraftslopereldGL

In [ ]:
map_lim = [-3000000,3000000]
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = dfmt.cut_domain_stereo(file_other, map_lim, map_lim)

In [ ]:
melt_2D

In [ ]:
box_charac_2D

In [ ]:
plt.figure()

for tt in tqdm(range(1980,1980 + 60)):
    
    # Thermal forcing
    T_S_2D_isfdraft = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS_'+str(tt)+'.nc').squeeze().drop('time')
    
    thermal_forcing = T_S_2D_isfdraft['theta_in'] - meltf.freezing_temperature(T_S_2D_isfdraft['salinity_in'],-file_other_cut['corrected_isfdraft'].sel(time=tt))
    
    # File isf
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    
    box_charac_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(tt)+'_merged75.nc')
    box_charac_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(tt)+'_merged75.nc')

    if box_charac_2D and box_charac_1D:
        box_loc_config2 = box_charac_2D['box_location'].sel(box_nb_tot=box_charac_1D['nD_config'].sel(config=2))
        box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=1).drop('Nisf')
    
    ## Ref melt
    #NEMO_melt_rates_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO_'+str(tt)+'.nc')
    #melt_rate = NEMO_melt_rates_2D['melt_m_ice_per_y']
    
    # NN_melt
    melt_2D = xr.open_dataset(outputpath_melt_nn + 'evalmetrics_2D_'+mod_size+'_'+exp_name+'_ensmean_'+TS_opt+'_norm'+norm_method+'_'+str(tt)+'_'+nemo_run+'.nc')
    melt_GL = melt_2D.where(np.isfinite(box1))

    plt.scatter(thermal_forcing.where(np.isfinite(box1)),melt_GL['reference_melt'],alpha=0.005,edgecolors=None,color='grey',s=5)

plt.title('REFERENCE')
    
plt.xlim(0,1.1)
plt.ylim(-2,3)

In [ ]:
plt.figure()

for tt in tqdm(range(1980,1980 + 60)):
    
    # Thermal forcing
    T_S_2D_isfdraft = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS_'+str(tt)+'.nc').squeeze().drop('time')
    
    thermal_forcing = T_S_2D_isfdraft['theta_in'] - meltf.freezing_temperature(T_S_2D_isfdraft['salinity_in'],-file_other_cut['corrected_isfdraft'].sel(time=tt))
    
    # File isf
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    
    box_charac_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(tt)+'_merged75.nc')
    box_charac_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(tt)+'_merged75.nc')

    if box_charac_2D and box_charac_1D:
        box_loc_config2 = box_charac_2D['box_location'].sel(box_nb_tot=box_charac_1D['nD_config'].sel(config=2))
        box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=1).drop('Nisf')
    
    ## Ref melt
    #NEMO_melt_rates_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO_'+str(tt)+'.nc')
    #melt_rate = NEMO_melt_rates_2D['melt_m_ice_per_y']
    
    # NN_melt
    melt_2D = xr.open_dataset(outputpath_melt_nn + 'evalmetrics_2D_'+mod_size+'_'+exp_name+'_ensmean_'+TS_opt+'_norm'+norm_method+'_'+str(tt)+'_'+nemo_run+'.nc')
    
    melt_GL = melt_2D.where(np.isfinite(box1))

    plt.scatter(thermal_forcing.where(np.isfinite(box1)),melt_GL['predicted_melt'],alpha=0.002,edgecolors=None,color='k',s=5)

plt.title('PREDICTED')

plt.xlim(0,1.1)
plt.ylim(-2,3)

ALL (NOT ONLY GL)

In [ ]:
plt.figure()

for tt in tqdm(range(1980,1980 + 60)):
    
    # Thermal forcing
    T_S_2D_isfdraft = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS_'+str(tt)+'.nc').squeeze().drop('time')
    
    thermal_forcing = T_S_2D_isfdraft['theta_in'] - meltf.freezing_temperature(T_S_2D_isfdraft['salinity_in'],-file_other_cut['corrected_isfdraft'].sel(time=tt))
    
    # File isf
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    
    
    ## Ref melt
    #NEMO_melt_rates_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO_'+str(tt)+'.nc')
    #melt_rate = NEMO_melt_rates_2D['melt_m_ice_per_y']
    
    # NN_melt
    melt_2D = xr.open_dataset(outputpath_melt_nn + 'evalmetrics_2D_'+mod_size+'_'+exp_name+'_ensmean_'+TS_opt+'_norm'+norm_method+'_'+str(tt)+'_'+nemo_run+'.nc')

    plt.scatter(thermal_forcing,melt_2D['reference_melt'],alpha=0.005,edgecolors=None,color='grey',s=5)

plt.title('REFERENCE (ALL POINTS)')
    
plt.xlim(0,1.1)
plt.ylim(-2,3)

LOOK AT DELTAS

In [ ]:
thermal_forcing_list = []
melt2D_list = []
for tt in tqdm(range(1980,1980 + 10)):
    
    # Thermal forcing
    T_S_2D_isfdraft = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS_'+str(tt)+'.nc').squeeze().drop('time')
    
    thermal_forcing = T_S_2D_isfdraft['theta_in'] - meltf.freezing_temperature(T_S_2D_isfdraft['salinity_in'],-file_other_cut['corrected_isfdraft'].sel(time=tt))
    
    # File isf
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    
    
    ## Ref melt
    #NEMO_melt_rates_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO_'+str(tt)+'.nc')
    #melt_rate = NEMO_melt_rates_2D['melt_m_ice_per_y']
    
    # NN_melt
    melt_2D = xr.open_dataset(outputpath_melt_nn + 'evalmetrics_2D_'+mod_size+'_'+exp_name+'_ensmean_'+TS_opt+'_norm'+norm_method+'_'+str(tt)+'_'+nemo_run+'.nc')
    
    thermal_forcing_list.append(thermal_forcing.assign_coords({'time': tt}))
    melt2D_list.append(melt_2D.assign_coords({'time': tt}))
    
    tf_early = xr.concat(thermal_forcing_list, dim='time')
    melt2D_early = xr.concat(melt2D_list, dim='time')


In [ ]:
thermal_forcing_list = []
melt2D_list = []
for tt in tqdm(range(2030,2030 + 10)):
    
    # Thermal forcing
    T_S_2D_isfdraft = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS_'+str(tt)+'.nc').squeeze().drop('time')
    
    thermal_forcing = T_S_2D_isfdraft['theta_in'] - meltf.freezing_temperature(T_S_2D_isfdraft['salinity_in'],-file_other_cut['corrected_isfdraft'].sel(time=tt))
    
    # File isf
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    
    
    ## Ref melt
    #NEMO_melt_rates_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO_'+str(tt)+'.nc')
    #melt_rate = NEMO_melt_rates_2D['melt_m_ice_per_y']
    
    # NN_melt
    melt_2D = xr.open_dataset(outputpath_melt_nn + 'evalmetrics_2D_'+mod_size+'_'+exp_name+'_ensmean_'+TS_opt+'_norm'+norm_method+'_'+str(tt)+'_'+nemo_run+'.nc')
    
    thermal_forcing_list.append(thermal_forcing.assign_coords({'time': tt}))
    melt2D_list.append(melt_2D.assign_coords({'time': tt}))
    
    tf_late = xr.concat(thermal_forcing_list, dim='time')
    melt2D_late = xr.concat(melt2D_list, dim='time')


In [ ]:
delta_melt

In [ ]:
delta_tf = tf_late.mean('time') - tf_early.mean('time')
delta_melt = melt2D_late.mean('time') - melt2D_early.mean('time')

In [ ]:
plt.figure()
plt.scatter(delta_tf,delta_melt['predicted_melt'],alpha=0.01,c='k',s=20)
plt.scatter(delta_tf,delta_melt['reference_melt'],alpha=0.01,c='grey',s=20)

In [ ]:
plt.figure()

for tt in tqdm(range(1980,1980 + 60)):
    
    # Thermal forcing
    T_S_2D_isfdraft = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS_'+str(tt)+'.nc').squeeze().drop('time')
    
    thermal_forcing = T_S_2D_isfdraft['theta_in'] - meltf.freezing_temperature(T_S_2D_isfdraft['salinity_in'],-file_other_cut['corrected_isfdraft'].sel(time=tt))
    
    # File isf
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    
    ## Ref melt
    #NEMO_melt_rates_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO_'+str(tt)+'.nc')
    #melt_rate = NEMO_melt_rates_2D['melt_m_ice_per_y']
    
    # NN_melt
    melt_2D = xr.open_dataset(outputpath_melt_nn + 'evalmetrics_2D_'+mod_size+'_'+exp_name+'_ensmean_'+TS_opt+'_norm'+norm_method+'_'+str(tt)+'_'+nemo_run+'.nc')
    
    plt.scatter(thermal_forcing,melt_2D['predicted_melt'],alpha=0.002,edgecolors=None,color='k',s=5)

plt.title('PREDICTED (ALL POINTS)')

plt.xlim(0,1.1)
plt.ylim(-2,3)

In [ ]:
melt_GL['predicted_melt'].plot()

Try KDE density plot

In [ ]:
x.flatten()

In [ ]:
x0 = thermal_forcing.values.flatten()
y0 = melt_2D['reference_melt'].values.flatten()

x1 = x0[~np.isnan(x0) & ~np.isnan(y0)]
y1 = y0[~np.isnan(x0) & ~np.isnan(y0)]

x = x1[(x1<=1) & (y1<=5)]
y = y1[(x1<=1) & (y1<=5)]

nbins=300
k = kde.gaussian_kde([x,y])
xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi = k(np.vstack([xi.flatten(), yi.flatten()]))
 
# Make the plot
plt.pcolormesh(xi, yi, zi.reshape(xi.shape), shading='auto', cmap=plt.cm.Greens_r)

In [ ]:
T_S_2D_isfdraft = xr.open_mfdataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS.nc').sel(profile_domain=50)

file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = file_other.sel(x=T_S_2D_isfdraft.x,y=T_S_2D_isfdraft.y)

In [ ]:
file_other_cut

In [ ]:
T_S_2D_isfdraft['salinity_in']

In [ ]:
thermal_forcing = T_S_2D_isfdraft['theta_in'] - meltf.freezing_temperature(T_S_2D_isfdraft['salinity_in'],-file_other_cut['corrected_isfdraft'])

In [ ]:
plt.